In [ ]:
! cd ../ && python setup.py develop

In [ ]:
import numpy as np
import pandas as pd
import logging
%load_ext autoreload
%autoreload 2

from mercari import tf_sparse

In [ ]:
small = False
df_train, df_test = tf_sparse.load_data(small=small)
df = pd.concat([df_train, df_test], 0)  # type: pd.DataFrame
nrow_train = df_train.shape[0]
y_train = np.log1p(df_train['price']).values
train_ids, valid_ids, submission = tf_sparse.load_valid_ids(df_train)

del df_train
del df_test
X, _, _, _ = tf_sparse.build_features(df)

scaler = tf_sparse.StandardScaler()
scaler.fit(y_train.reshape(-1 , 1));

In [ ]:
X_train = X[:nrow_train]
train_nonzero = (df['price'].iloc[train_ids] != 0).values.nonzero()[0]
predictions = []
for i in range(3):
    model, model_predictions = tf_sparse.train_model(
        i, scaler,
        X_train[train_ids][train_nonzero],
        y_train[train_ids][train_nonzero],
        X_train[valid_ids])
    y_pred_valid = tf_sparse.weighted_predictions(model_predictions)
    predictions.append(y_pred_valid)
    rmsle = tf_sparse.get_rmsle(y_pred_valid, y_train[valid_ids])
    logging.info(f'rmsle: {rmsle:.5f}')
    submission['price'] = np.expm1(y_pred_valid)
    submission.to_csv(f'valid_{type(model).__name__}_{i}.csv', index=False)
logging.info('rmsle of mean: {:.5f}'.format(
    tf_sparse.get_rmsle(np.mean(predictions, 0), y_train[valid_ids])))